In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

In [3]:
faults_diagnostics = pd.read_csv('faults_diagnostics.csv')
target = pd.read_csv('target.csv')

In [4]:
faults_diagnostics['EventTimeStamp'] = pd.to_datetime(faults_diagnostics['EventTimeStamp'])

In [5]:
faults_diagnostics = faults_diagnostics.loc[faults_diagnostics['active'] == True]

In [6]:
faults_diagnostics.columns

Index(['Unnamed: 0', 'FaultId', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake', 'Speed',
       'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure', 'RecordID',
       'ESS_Id', 'EventTimeStamp', 'eventDescription', 'ecuSoftwareVersion',
       'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'spn', 'fmi',
       'active', 'activeTransitionCount', 'EquipmentID', 'MCTNumber',
       'Latitude', 'Longitude', 'LocationTimeStamp'],
      dtype='object')

In [8]:
faults_diagnostics_Final = target.merge(faults_diagnostics, left_on = 'RecordID', right_on = 'FaultId')

In [9]:
faults_diagnostics_Final['spnFmi'] = faults_diagnostics_Final['spn'].astype(str) + '-' + faults_diagnostics_Final['fmi'].astype(str)

In [ ]:
#dropped Columns in the below cell

In [10]:
faults_diagnostics_Final = faults_diagnostics_Final.drop(columns = ['spn', 'fmi', 'FaultId', 'Unnamed: 0', 'RecordID_x', 'ecuModel', 'LocationTimeStamp', 'CruiseControlSetSpeed', 'RecordID_y', 'SwitchedBatteryVoltage', 'Throttle', 'ParkingBrake', 'ecuSoftwareVersion', 'ecuSerialNumber', 'LampStatus', 'MCTNumber', 'eventDescription', 'active', 'ESS_Id'], axis = 1)

In [11]:
faults_diagnostics_Final['Merge_ID'] = 1

In [13]:
faults_diagnostics_Final = pd.get_dummies(faults_diagnostics_Final, columns = ['CruiseControlActive', 'IgnStatus', 'ecuMake', 'spnFmi'], prefix=['CruiseControlActive', 'IgnStatus', 'ecuMake', 'spnFmi'])
faults_diagnostics_Final

,target,AcceleratorPedal,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,...,spnFmi_96-3,spnFmi_96-4,spnFmi_96-9,spnFmi_97-15,spnFmi_97-16,spnFmi_97-3,spnFmi_97-4,spnFmi_976-9,spnFmi_98-18,spnFmi_98-5
0,False,0.0,14.2100,423178.70000,100.4,11.0,0.00,96.74375,0.000,1632.20,...,0,0,0,0,0,0,0,0,0,0
1,False,9.2,14.3550,368419.30000,181.4,18.0,36.54,201.08750,1008.000,6960.80,...,0,0,0,0,0,0,0,0,0,0
2,False,9.2,14.3550,368419.30000,181.4,18.0,36.54,201.08750,1008.000,6960.80,...,0,0,0,0,0,0,0,0,0,0
3,False,48.0,14.4275,470381.40000,181.4,30.0,38.28,196.53130,1514.500,9480.00,...,0,0,0,0,0,0,0,0,0,0
4,False,82.8,14.2825,278736.70000,188.6,80.0,39.44,210.03130,1711.375,6292.20,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546415,False,0.0,14.6450,391932.60000,181.4,11.0,22.62,197.60000,597.375,8016.75,...,1,0,0,0,0,0,0,0,0,0
546416,False,0.0,14.3550,457529.70000,181.4,11.0,19.72,207.21880,600.250,13047.05,...,0,0,0,0,0,0,0,0,0,0
546417,False,100.0,14.5000,423937.90000,185.0,51.0,37.12,211.49370,1310.250,10722.70,...,0,0,0,0,0,0,0,0,0,0
546418,False,0.0,14.3550,465925.40000,186.8,62.0,41.18,212.84380,1340.750,9326.75,...,0,0,0,0,0,0,0,0,0,0


In [14]:
newlist_1 = [x for x in faults_diagnostics_Final.columns.tolist() if "spnFmi" in x]

In [16]:
newlist_1.append('EquipmentID')
newlist_1.append('Merge_ID')
newlist_1.append('EventTimeStamp')

In [17]:
newlist

['spnFmi_0-0',
 'spnFmi_100-1',
 'spnFmi_100-18',
 'spnFmi_100-2',
 'spnFmi_100-3',
 'spnFmi_100-4',
 'spnFmi_101-0',
 'spnFmi_101-15',
 'spnFmi_101-16',
 'spnFmi_101-2',
 'spnFmi_101-3',
 'spnFmi_101-4',
 'spnFmi_102-10',
 'spnFmi_102-14',
 'spnFmi_102-15',
 'spnFmi_102-16',
 'spnFmi_102-17',
 'spnFmi_102-18',
 'spnFmi_102-2',
 'spnFmi_102-20',
 'spnFmi_102-3',
 'spnFmi_102-31',
 'spnFmi_102-4',
 'spnFmi_102-5',
 'spnFmi_1023-5',
 'spnFmi_1024-0',
 'spnFmi_1028-9',
 'spnFmi_103-1',
 'spnFmi_103-10',
 'spnFmi_103-16',
 'spnFmi_103-18',
 'spnFmi_103-4',
 'spnFmi_103-7',
 'spnFmi_103-9',
 'spnFmi_1043-2',
 'spnFmi_1045-2',
 'spnFmi_1045-7',
 'spnFmi_105-0',
 'spnFmi_105-17',
 'spnFmi_105-18',
 'spnFmi_105-2',
 'spnFmi_105-3',
 'spnFmi_105-5',
 'spnFmi_1056-2',
 'spnFmi_1056-4',
 'spnFmi_1056-5',
 'spnFmi_1059-2',
 'spnFmi_1067-11',
 'spnFmi_1067-2',
 'spnFmi_1067-7',
 'spnFmi_1068-2',
 'spnFmi_107-3',
 'spnFmi_107-4',
 'spnFmi_1071-5',
 'spnFmi_1072-1',
 'spnFmi_1072-3',
 'spnFmi_1072-5'

In [18]:
faults_diagnostics_Final = faults_diagnostics_Final.sort_values('EventTimeStamp', ascending = True)

In [19]:
faults_diagnostics_rolling_1 = faults_diagnostics_Final[newlist].groupby('EquipmentID').rolling('5D', on = 'EventTimeStamp').sum()

In [20]:
faults_diagnostics_rolling_1

Merge_ID  spnFmi_0-0  spnFmi_100-1  spnFmi_100-18  \
EquipmentID                                                             
301         24464        1.0         0.0           0.0            0.0   
            25357        2.0         0.0           0.0            0.0   
            27715        1.0         0.0           0.0            0.0   
            29792        2.0         0.0           0.0            0.0   
            33526        1.0         0.0           0.0            0.0   
...                      ...         ...           ...            ...   
2377        545568       1.0         0.0           0.0            0.0   
            545942       2.0         0.0           0.0            0.0   
            546419       2.0         0.0           0.0            0.0   
2380        546080       1.0         0.0           0.0            0.0   
2381        546401       1.0         0.0           0.0            0.0   

                    spnFmi_100-2  spnFmi_100-3  spnFmi_100-4  spnFmi_101-0  \
EquipmentID                                                                  
301         24464            0.0           0.0           0.0           0.0   
            25357            0.0           0.0           0.0           0.0   
            27715            0.0           0.0           0.0           0.0   
            29792            0.0           0.0           0.0           0.0   
            33526            0.0           0.0           0.0           0.0   
...                          ...           ...           ...           ...   
2377        545568           0.0           0.0           0.0           0.0   
            545942           0.0           0.0           0.0           0.0   
            546419           0.0           0.0           0.0           0.0   
2380        546080           0.0           0.0           0.0           0.0   
2381        546401           0.0           0.0           0.0           0.0   

                    spnFmi_101-15  spnFmi_101-16  ...  spnFmi_96-4  \
EquipmentID                                       ...                
301         24464             0.0            0.0  ...          0.0   
            25357             0.0            0.0  ...          0.0   
            27715             0.0            0.0  ...          0.0   
            29792             0.0            0.0  ...          0.0   
            33526             0.0            0.0  ...          0.0   
...                           ...            ...  ...          ...   
2377        545568            0.0            0.0  ...          0.0   
            545942            0.0            0.0  ...          0.0   
            546419            0.0            0.0  ...          0.0   
2380        546080            0.0            0.0  ...          0.0   
2381        546401            0.0            0.0  ...          0.0   

                    spnFmi_96-9  spnFmi_97-15  spnFmi_97-16  spnFmi_97-3  \
EquipmentID                                                                
301         24464           0.0           0.0           0.0          0.0   
            25357           0.0           0.0           0.0          0.0   
            27715           0.0           0.0           0.0          0.0   
            29792           0.0           0.0           0.0          0.0   
            33526           0.0           0.0           0.0          0.0   
...                         ...           ...           ...          ...   
2377        545568          0.0           0.0           0.0          0.0   
            545942          0.0           0.0           0.0          0.0   
            546419          0.0           0.0           0.0          0.0   
2380        546080          0.0           0.0           0.0          0.0   
2381        546401          0.0           0.0           0.0          0.0   

                    spnFmi_97-4  spnFmi_976-9  spnFmi_98-18  \
EquipmentID                                                   
301         24464           0

In [21]:
new_list_2 = [x for x in faults_diagnostics_Final.columns.tolist() if "spnFmi" in x]

In [22]:
faults_diagnostics_Final = faults_diagnostics_Final.drop(columns = new_list_2)

In [23]:
dropped_columns = faults_diagnostics_Final.drop(columns = ['Merge_ID', 'EquipmentID', 'EventTimeStamp'])

In [24]:
agg_dict = {
    x : lambda x: x[-1] for x in dropped_columns.columns.tolist()
}

In [25]:
agg_dict['Merge_ID'] = 'sum'

In [27]:
test_2 = (faults_diagnostics_Final.groupby('EquipmentID').rolling('5D', on = 'EventTimeStamp').agg(agg_dict))

In [28]:
faults_diagnostics_rolling_2 = faults_diagnostics_rolling_1.merge(test_2, on = ['EquipmentID', 'EventTimeStamp', 'Merge_ID'], how = 'inner')

In [29]:
target_1 = faults_diagnostics_rolling_2[['target']]

In [36]:
target_1 = target_1.reset_index()

In [32]:
Predictors = faults_diagnostics_rolling_2.drop(columns = ['target', 'EventTimeStamp', 'Merge_ID'])

In [38]:
Predictors = Predictors.reset_index()

In [40]:
Predictors.to_csv('Predictors.csv', index = True)

In [41]:
target_1.to_csv('target_1.csv', index = True)

In [39]:
import random

trucks = target_1[['EquipmentID']].drop_duplicates()['EquipmentID'].to_list()

random.shuffle(trucks)

trainTrucks = trucks[0:831]

testTrucks = trucks[831:]

xTrain = Predictors.loc[Predictors['EquipmentID'].isin(trainTrucks)].drop(columns = 'EquipmentID')
xTest = Predictors.loc[Predictors['EquipmentID'].isin(testTrucks)].drop(columns = 'EquipmentID')
yTrain = target_1.loc[target_1['EquipmentID'].isin(trainTrucks)].drop(columns = 'EquipmentID')
yTest = target_1.loc[target_1['EquipmentID'].isin(testTrucks)].drop(columns = 'EquipmentID')

MemoryError: Unable to allocate 3.31 GiB for an array with shape (1009, 439805) and data type float64

In [22]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1049319 entries, 1025117 to 1054112
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   FaultId                    1049319 non-null  int64         
 1   AcceleratorPedal           1049319 non-null  float64       
 2   BarometricPressure         1049319 non-null  float64       
 3   CruiseControlActive        1049319 non-null  object        
 4   CruiseControlSetSpeed      517914 non-null   float64       
 5   DistanceLtd                1049319 non-null  float64       
 6   EngineCoolantTemperature   1049319 non-null  float64       
 7   EngineLoad                 1049319 non-null  float64       
 8   EngineOilPressure          1049319 non-null  float64       
 9   EngineOilTemperature       1049319 non-null  float64       
 10  EngineRpm                  1049319 non-null  float64       
 11  EngineTimeLtd              1049

In [34]:
faults_diagnostics

,Unnamed: 0,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,...,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,0,1,0.0,14.2100,False,66.48672,423178.70000,100.4,11.0,0.00,...,0,111,17,True,2.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,1,2,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,11,629,12,True,127.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
3,3,4,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,...,11,1807,2,True,127.0,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
5,5,6,48.0,14.4275,False,64.62260,470381.40000,181.4,30.0,38.28,...,0,111,17,True,1.0,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
6,6,7,82.8,14.2825,False,64.62260,278736.70000,188.6,80.0,39.44,...,0,111,17,True,2.0,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054103,1054103,1248448,0.0,14.6450,False,66.48672,391932.60000,181.4,11.0,22.62,...,49,96,3,True,126.0,1936,105355619,30.376851,-81.744953,2020-03-06 13:29:33.000
1054106,1054106,1248452,0.0,14.3550,False,66.48672,457529.70000,181.4,11.0,19.72,...,0,111,18,True,93.0,1886,105351219,39.015694,-77.031157,2020-03-06 13:43:24.000
1054109,1054109,1248455,100.0,14.5000,True,64.62260,423937.90000,185.0,51.0,37.12,...,0,1569,31,True,5.0,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1054110,1054110,1248456,0.0,14.3550,True,66.48672,465925.40000,186.8,62.0,41.18,...,0,3216,10,True,1.0,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
